# Inference on SLOSH + naip (houston) using Pix2PixHD trained on Masks_v1_sloshed [Bjorn]

In [2]:
#!pip install dominate ; pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [6]:
# Load data into instance

CP_PATH="data/eie_floods/houston/"
mkdir -p /home/jupyter/eie_vision/$CP_PATH/test_SLOSH
mkdir -p /home/jupyter/eie_vision/$CP_PATH/test_A
gsutil -m cp -r gs://eie_floods/naip/houston/tiles/* /home/jupyter/eie_vision/$CP_PATH/test_A/
gsutil -m cp -r gs://eie_floods/slosh/texas_to_maine/tiles/* /home/jupyter/eie_vision/$CP_PATH/test_SLOSH/

rm /home/jupyter/temp/Pix2pixHD/conditional_binary_sloshed/test_latest/images/US_*
rm /home/jupyter/temp/Pix2pixHD/conditional_binary_sloshed/test_latest/to_segment/US_*
#rm /home/jupyter/temp/Pix2pixHD/conditional_binary_sloshed_houston


Copying gs://eie_floods/naip/houston/tiles/./US_Category_5_MOM_Inundation_HighTide_00000000_pre_disaster.png...
Copying gs://eie_floods/naip/houston/tiles/./US_Category_5_MOM_Inundation_HighTide_00000002_pre_disaster.png...
Copying gs://eie_floods/naip/houston/tiles/./US_Category_5_MOM_Inundation_HighTide_00000001_pre_disaster.png...
Copying gs://eie_floods/naip/houston/tiles/./US_Category_5_MOM_Inundation_HighTide_00000004_pre_disaster.png...
Copying gs://eie_floods/naip/houston/tiles/./US_Category_5_MOM_Inundation_HighTide_00000003_pre_disaster.png...
Copying gs://eie_floods/naip/houston/tiles/./US_Category_5_MOM_Inundation_HighTide_00000005_pre_disaster.png...
Copying gs://eie_floods/naip/houston/tiles/./US_Category_5_MOM_Inundation_HighTide_00000006_pre_disaster.png...
Copying gs://eie_floods/naip/houston/tiles/./US_Category_5_MOM_Inundation_HighTide_00000007_pre_disaster.png...
Copying gs://eie_floods/naip/houston/tiles/./US_Category_5_MOM_Inundation_HighTide_00000009_pre_disaster

In [10]:
#Generate predictions over the test set. Only first 150 samples. We do not have SLOSH for Hurricane Matthew
#!cd ../.. ; python src/models/Pix2pixHD/test.py --checkpoints_dir pretrained/Pix2pixHD/  --phase test --how_many 2000 --name conditional_binary_sloshed --dataroot xBD --dataset_mode physics_aligned_bin_sloshed --no_instance --label_nc 0 --input_nc 4 --gpu_ids 7 --results_dir temp/Pix2pixHD/
!cd ../.. ; python src/models/Pix2pixHD/test.py --checkpoints_dir pretrained/Pix2pixHD/  --phase test --how_many 2000 --name conditional_binary_sloshed --dataroot data/eie_floods/houston --slosh_inference --dataset_mode physics_aligned_bin_sloshed --no_instance --label_nc 0 --input_nc 4 --gpu_ids 7 --results_dir temp/Pix2pixHD

------------ Options -------------
aspect_ratio: 1.0
augmentation_aggressive: False
batchSize: 1
checkpoints_dir: pretrained/Pix2pixHD/
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: data/eie_floods/houston
dataset_mode: physics_aligned_bin_sloshed
display_winsize: 512
engine: None
export_onnx: None
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [7]
how_many: 2000
input_nc: 4
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
model: pix2pixHD
nThreads: 8
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_local_enhancers: 1
name: conditional_binary_sloshed
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: True
norm: instance
normD: spectral
ntest: inf
onnx: None
output_nc: 3
phase: test
resize_or_crop: scale_width
results_dir: temp/Pix2pixHD
serial_batches: False
slosh_inference: True
tf_log: False
use_dropout: Fa

### We move things around at the moment in ./Temp/
The model output is placed into ./temp/ .We use this directory to work, then we carefully selects what goes into ./results/.
./temp/ does not get uploaded into our bucket, only ./results/ do!

In [11]:
# Move synthesized results to a folder to be used by the segmentation network
!cd ../../temp/Pix2pixHD/conditional_binary_sloshed/test_latest/ ; mkdir to_segment; cp -R images/US_Category*synthesized*.jpg to_segment

mkdir: cannot create directory ‘to_segment’: File exists


In [12]:
#Create Segmentation Masks
#Generate masks for all of the data generated by pix2pixHD_baseline
# TODO: fix loses.cvs error
! cd ../.. ; python src/models/Pix2pix-CycleGAN/test.py --phase to_segment --dataroot temp/Pix2pixHD/conditional_binary_sloshed/test_latest/ --direction AtoB --results_dir ./temp/Pix2pixHD/conditional_binary_sloshed_masks --dataset_mode single --model pix2pix --checkpoints_dir ./pretrained/Pix2pix-CycleGAN/flood_segmentation/scratch_1024_plus/ --name . --num_test 2000 --no_flip --gpu_ids 1 --max_dataset_size 2000 --batch_size 1  --load_size 1024 --crop_size 1024

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./pretrained/Pix2pix-CycleGAN/flood_segmentation/scratch_1024_plus/	[default: ./checkpoints]
                crop_size: 1024                          	[default: 256]
                 dataroot: temp/Pix2pixHD/conditional_binary_sloshed/test_latest/	[default: None]
             dataset_mode: single                        	[default: aligned]
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 1                             	[default: 0]
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                           

### Copy generated images into the right ./Results/. directory

In [13]:
mkdir -p /home/jupyter/eie_vision/results/Pix2pixHD/conditional_binary_sloshed_houston/

In [14]:
! cd ../.. ; cp -R temp/Pix2pixHD/conditional_binary_sloshed/test_latest/images/* results/Pix2pixHD/conditional_binary_sloshed_houston/ ; cp -R temp/Pix2pixHD/conditional_binary_sloshed_masks/to_segment_latest/images/*image_fake_B.png results/Pix2pixHD/conditional_binary_sloshed_houston/

## Copy generated images into the bucket

In [15]:
#Rename generated imagery
! cd /home/jupyter/eie_vision/results/Pix2pixHD/conditional_binary_sloshed_houston; rename 's/_pre_disaster_synthesized_image.jpg/_pre_disaster.jpg/g' *


In [39]:
!gsutil -m cp -r  /home/jupyter/eie_vision/results/Pix2pixHD/conditional_binary_sloshed_houston/*[0-9].jpg gs://eie_floods/Pix2pixHD/conditional_binary/houston/tiles

Copying file:///home/jupyter/eie_vision/results/Pix2pixHD/conditional_binary_sloshed_houston/US_Category_5_MOM_Inundation_HighTide_00000000.jpg [Content-Type=image/jpeg]...
Copying file:///home/jupyter/eie_vision/results/Pix2pixHD/conditional_binary_sloshed_houston/US_Category_5_MOM_Inundation_HighTide_00000001.jpg [Content-Type=image/jpeg]...
Copying file:///home/jupyter/eie_vision/results/Pix2pixHD/conditional_binary_sloshed_houston/US_Category_5_MOM_Inundation_HighTide_00000003.jpg [Content-Type=image/jpeg]...
Copying file:///home/jupyter/eie_vision/results/Pix2pixHD/conditional_binary_sloshed_houston/US_Category_5_MOM_Inundation_HighTide_00000002.jpg [Content-Type=image/jpeg]...
Copying file:///home/jupyter/eie_vision/results/Pix2pixHD/conditional_binary_sloshed_houston/US_Category_5_MOM_Inundation_HighTide_00000004.jpg [Content-Type=image/jpeg]...
Copying file:///home/jupyter/eie_vision/results/Pix2pixHD/conditional_binary_sloshed_houston/US_Category_5_MOM_Inundation_HighTide_0000